# Energy Efficiency Prediction using Linear Regression

This notebook implements Linear Regression for predicting building energy efficiency (heating load Y1 and cooling load Y2).

## Dataset Features:
- **X1**: Relative Compactness
- **X2**: Surface Area (m²)
- **X3**: Wall Area (m²)
- **X4**: Roof Area (m²)
- **X5**: Overall Height (m)
- **X6**: Orientation (2-5)
- **X7**: Glazing Area (0-0.4)
- **X8**: Glazing Area Distribution (0-5)

## Targets:
- **Y1**: Heating Load (kWh/m²)
- **Y2**: Cooling Load (kWh/m²)

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully!")

## 📚 Thư viện và Cấu hình

**Mục đích:** Import các thư viện cần thiết cho phân tích Machine Learning

**Thư viện chính:**
- `numpy`, `pandas`: Xử lý dữ liệu và tính toán
- `matplotlib`, `seaborn`: Trực quan hóa dữ liệu
- `sklearn`: Thuật toán Machine Learning, preprocessing, metrics
- `scipy.stats`: Kiểm định thống kê (Q-Q plot)

## 1. Data Loading and Exploration

**Công việc:** 
- Load dữ liệu từ file CSV
- Loại bỏ cột rỗng
- Phân tích cấu trúc: shape, data types, missing values, statistics

In [ ]:
# Load the dataset
df = pd.read_csv("ENB2012_data.csv", index_col=0)


# Display basic information
print("Dataset Shape:", df.shape)
print("\n" + "="*70)
print("First few rows:")
print(df.head(10))
print("\n" + "="*70)
print("\nDataset Info:")
print(df.info())
print("\n" + "="*70)
print("\nStatistical Summary:")
print(df.describe())
print("\n" + "="*70)
print("\nMissing Values:")
print(df.isnull().sum())

Dataset Shape: (768, 11)

First few rows:
   Unnamed: 0    X1     X2     X3      X4   X5  X6   X7  X8     Y1     Y2
0           1  0.98  514.5  294.0  110.25  7.0   2  0.0   0  15.55  21.33
1           2  0.98  514.5  294.0  110.25  7.0   3  0.0   0  15.55  21.33
2           3  0.98  514.5  294.0  110.25  7.0   4  0.0   0  15.55  21.33
3           4  0.98  514.5  294.0  110.25  7.0   5  0.0   0  15.55  21.33
4           5  0.90  563.5  318.5  122.50  7.0   2  0.0   0  20.84  28.28
5           6  0.90  563.5  318.5  122.50  7.0   3  0.0   0  21.46  25.38
6           7  0.90  563.5  318.5  122.50  7.0   4  0.0   0  20.71  25.16
7           8  0.90  563.5  318.5  122.50  7.0   5  0.0   0  19.68  29.60
8           9  0.86  588.0  294.0  147.00  7.0   2  0.0   0  19.50  27.30
9          10  0.86  588.0  294.0  147.00  7.0   3  0.0   0  19.95  21.97


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 11 columns):
 #   Column      Non-Nu

## 2. Data Preparation

**Bước chuẩn bị dữ liệu:**

1. **Tách features và targets:** 
   - Features (X): 8 biến độc lập (X1-X8)
   - Targets: Y1 (Heating Load), Y2 (Cooling Load)

2. **Train-Test Split (80-20):**
   - Training set: 80% dữ liệu để huấn luyện mô hình
   - Testing set: 20% dữ liệu để đánh giá mô hình

3. **Feature Scaling (Standardization):**
   
   $$z = \frac{x - \mu}{\sigma}$$
   
   Trong đó:
   - $x$: giá trị gốc
   - $\mu$: trung bình (mean)
   - $\sigma$: độ lệch chuẩn (standard deviation)
   - $z$: giá trị chuẩn hóa
   
   **Lợi ích:** Đưa tất cả features về cùng scale, giúp model hội tụ nhanh hơn

In [ ]:
# Separate features and targets
X = df.iloc[:, :-2].values  # X1 to X8 features
y1 = df['Y1'].values  # Heating Load
y2 = df['Y2'].values  # Cooling Load

# Feature names for later use
feature_names = ['X1_Compactness', 'X2_SurfaceArea', 'X3_WallArea', 
                 'X4_RoofArea', 'X5_Height', 'X6_Orientation', 
                 'X7_GlazingArea', 'X8_GlazingDist']

print("Features shape:", X.shape)
print("Y1 (Heating Load) shape:", y1.shape)
print("Y2 (Cooling Load) shape:", y2.shape)

# Split data into training and testing sets (80-20 split)
# Using stratified approach would be ideal but for regression we use random split
X_train, X_test, y1_train, y1_test = train_test_split(
    X, y1, test_size=0.2, random_state=42
)
X_train2, X_test2, y2_train, y2_test = train_test_split(
    X, y2, test_size=0.2, random_state=42
)

print("\nTraining set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])

# Feature Scaling (Standardization)
# Standardization is crucial for Linear Regression with multiple features
# Formula: z = (x - μ) / σ
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# For Y2 we use the same split (same X features)
X_train2_scaled = X_train_scaled
X_test2_scaled = X_test_scaled

print("\n✓ Data preparation completed!")
print(f"Feature means after scaling (should be ~0): {X_train_scaled.mean(axis=0)}")
print(f"Feature stds after scaling (should be ~1): {X_train_scaled.std(axis=0)}")

In [ ]:
# Correlation Analysis
print("="*70)
print("CORRELATION ANALYSIS")
print("="*70)

# Create correlation matrix
df_features = df.copy()
correlation_matrix = df_features.corr()

# Correlation with targets
print("\nCorrelation with Y1 (Heating Load):")
y1_corr = correlation_matrix['Y1'].drop('Y1').sort_values(ascending=False)
print(y1_corr)

print("\nCorrelation with Y2 (Cooling Load):")
y2_corr = correlation_matrix['Y2'].drop('Y2').sort_values(ascending=False)
print(y2_corr)

# Visualize correlation matrix
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Full correlation matrix
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, ax=axes[0], cbar_kws={'shrink': 0.8})
axes[0].set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold')

# Correlation with targets only
target_corr = pd.DataFrame({
    'Y1': correlation_matrix['Y1'].drop(['Y1', 'Y2']),
    'Y2': correlation_matrix['Y2'].drop(['Y1', 'Y2'])
})
sns.heatmap(target_corr, annot=True, fmt='.2f', cmap='RdYlGn', 
            center=0, ax=axes[1], cbar_kws={'shrink': 0.8})
axes[1].set_title('Features Correlation with Targets', fontsize=14, fontweight='bold')
axes[1].set_xlabel('')

plt.tight_layout()
plt.show()

print("\n✓ Correlation analysis completed!")

### 2.1 Feature Correlation Analysis

Phân tích correlation giúp hiểu mối quan hệ giữa features và targets.

## 3. Linear Regression Model Training

**Thuật toán Linear Regression:**

Mô hình dự đoán dựa trên phương trình tuyến tính:

$$\hat{y} = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n$$

Hoặc dạng vector:

$$\hat{y} = \beta_0 + \mathbf{X}\boldsymbol{\beta}$$

Trong đó:
- $\hat{y}$: giá trị dự đoán
- $\beta_0$: intercept (hằng số)
- $\beta_1, \beta_2, ..., \beta_n$: coefficients (hệ số)
- $x_1, x_2, ..., x_n$: features (biến độc lập)

**Phương pháp tối ưu:** Ordinary Least Squares (OLS)

$$\min_{\boldsymbol{\beta}} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

**Nghiệm giải tích:**

$$\boldsymbol{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

In [ ]:
# Train Linear Regression for Y1 (Heating Load)
print("Training Linear Regression Model for Y1 (Heating Load)...")
model_y1 = LinearRegression()
model_y1.fit(X_train_scaled, y1_train)

# Predictions for Y1
y1_pred_train = model_y1.predict(X_train_scaled)
y1_pred_test = model_y1.predict(X_test_scaled)

print("✓ Y1 Model trained successfully!")
print(f"  Intercept: {model_y1.intercept_:.4f}")
print(f"  Number of features: {len(model_y1.coef_)}")
print(f"  Coefficient range: [{model_y1.coef_.min():.4f}, {model_y1.coef_.max():.4f}]")

# Train Linear Regression for Y2 (Cooling Load)
print("\nTraining Linear Regression Model for Y2 (Cooling Load)...")
model_y2 = LinearRegression()
model_y2.fit(X_train2_scaled, y2_train)

# Predictions for Y2
y2_pred_train = model_y2.predict(X_train2_scaled)
y2_pred_test = model_y2.predict(X_test2_scaled)

print("✓ Y2 Model trained successfully!")
print(f"  Intercept: {model_y2.intercept_:.4f}")
print(f"  Number of features: {len(model_y2.coef_)}")
print(f"  Coefficient range: [{model_y2.coef_.min():.4f}, {model_y2.coef_.max():.4f}]")

# Quick performance check on training set (to detect overfitting)
train_r2_y1 = r2_score(y1_train, y1_pred_train)
train_r2_y2 = r2_score(y2_train, y2_pred_train)
print(f"\nTraining R² scores:")
print(f"  Y1: {train_r2_y1:.4f}")
print(f"  Y2: {train_r2_y2:.4f}")

## 4. Model Evaluation Metrics

**Các chỉ số đánh giá hiệu suất mô hình:**

1. **R² (Coefficient of Determination):**
   
   $$R^2 = 1 - \frac{SS_{res}}{SS_{tot}} = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}$$
   
   - Giá trị: 0 đến 1 (càng cao càng tốt)
   - Ý nghĩa: % phương sai của y được giải thích bởi mô hình
   - $R^2 = 1$: dự đoán hoàn hảo
   - $R^2 = 0$: mô hình không tốt hơn việc dự đoán theo trung bình

2. **RMSE (Root Mean Squared Error):**
   
   $$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}$$
   
   - Giá trị: ≥ 0 (càng thấp càng tốt)
   - Ý nghĩa: Độ lệch trung bình giữa giá trị thực và dự đoán
   - Nhạy với outliers (do bình phương sai số)

3. **MAE (Mean Absolute Error):**
   
   $$MAE = \frac{1}{n}\sum_{i=1}^{n}|y_i - \hat{y}_i|$$
   
   - Giá trị: ≥ 0 (càng thấp càng tốt)
   - Ý nghĩa: Sai số tuyệt đối trung bình
   - Ít nhạy với outliers hơn RMSE

In [ ]:
# Function to calculate and display metrics
def evaluate_model(y_true, y_pred, dataset_name, target_name):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    print(f"\n{'='*70}")
    print(f"{target_name} - {dataset_name} Set Metrics:")
    print(f"{'='*70}")
    print(f"R² (Coefficient of Determination): {r2:.6f}")
    print(f"RMSE (Root Mean Squared Error):    {rmse:.6f}")
    print(f"MAE (Mean Absolute Error):          {mae:.6f}")
    
    return {'R2': r2, 'RMSE': rmse, 'MAE': mae}

# Evaluate Y1 Model
print("\n" + "="*70)
print("Y1 MODEL EVALUATION (HEATING LOAD)")
print("="*70)
y1_train_metrics = evaluate_model(y1_train, y1_pred_train, "Training", "Y1")
y1_test_metrics = evaluate_model(y1_test, y1_pred_test, "Testing", "Y1")

# Evaluate Y2 Model
print("\n" + "="*70)
print("Y2 MODEL EVALUATION (COOLING LOAD)")
print("="*70)
y2_train_metrics = evaluate_model(y2_train, y2_pred_train, "Training", "Y2")
y2_test_metrics = evaluate_model(y2_test, y2_pred_test, "Testing", "Y2")

# Create comparison table
print("\n" + "="*70)
print("SUMMARY COMPARISON TABLE")
print("="*70)
comparison_df = pd.DataFrame({
    'Y1_Train': [y1_train_metrics['R2'], y1_train_metrics['RMSE'], y1_train_metrics['MAE']],
    'Y1_Test': [y1_test_metrics['R2'], y1_test_metrics['RMSE'], y1_test_metrics['MAE']],
    'Y2_Train': [y2_train_metrics['R2'], y2_train_metrics['RMSE'], y2_train_metrics['MAE']],
    'Y2_Test': [y2_test_metrics['R2'], y2_test_metrics['RMSE'], y2_test_metrics['MAE']]
}, index=['R²', 'RMSE', 'MAE'])

print(comparison_df)

In [ ]:
# Overfitting/Underfitting Analysis
print("="*70)
print("OVERFITTING/UNDERFITTING ANALYSIS")
print("="*70)

# Calculate train-test gaps
for target_name, train_pred, test_pred, y_train_true, y_test_true in [
    ('Y1', y1_pred_train, y1_pred_test, y1_train, y1_test),
    ('Y2', y2_pred_train, y2_pred_test, y2_train, y2_test)
]:
    train_r2 = r2_score(y_train_true, train_pred)
    test_r2 = r2_score(y_test_true, test_pred)
    r2_gap = train_r2 - test_r2
    
    train_rmse = np.sqrt(mean_squared_error(y_train_true, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test_true, test_pred))
    rmse_gap = test_rmse - train_rmse
    
    print(f"\n{target_name} Analysis:")
    print(f"  Train R²: {train_r2:.6f}")
    print(f"  Test R²:  {test_r2:.6f}")
    print(f"  R² Gap:   {r2_gap:.6f} {'⚠️ (possible overfitting)' if r2_gap > 0.05 else '✓ (good)'}")
    
    print(f"  Train RMSE: {train_rmse:.6f}")
    print(f"  Test RMSE:  {test_rmse:.6f}")
    print(f"  RMSE Gap:   {rmse_gap:.6f} {'⚠️ (possible overfitting)' if rmse_gap > 1.0 else '✓ (good)'}")
    
    # Diagnosis
    if train_r2 > 0.95 and test_r2 > 0.95 and r2_gap < 0.02:
        status = "✓ EXCELLENT FIT - No overfitting detected"
    elif train_r2 > 0.90 and test_r2 > 0.85 and r2_gap < 0.05:
        status = "✓ GOOD FIT - Model generalizes well"
    elif r2_gap > 0.1:
        status = "⚠️ WARNING - Possible overfitting"
    elif train_r2 < 0.7 and test_r2 < 0.7:
        status = "⚠️ WARNING - Underfitting (model too simple)"
    else:
        status = "~ ACCEPTABLE FIT"
    
    print(f"  Status: {status}")

print("\n" + "="*70)

### 4.1 Overfitting/Underfitting Analysis

**Kiểm tra hiện tượng:**
- **Overfitting:** Training score >> Test score (model học thuộc lòng data)
- **Underfitting:** Cả Training và Test scores đều thấp (model quá đơn giản)
- **Good fit:** Training score ≈ Test score và cả hai đều cao

## 5. Cross-Validation Analysis

**K-Fold Cross-Validation:**

Kỹ thuật đánh giá mô hình đáng tin cậy hơn train-test split đơn thuần.

**Quy trình:**
1. Chia dataset thành K folds (K=10)
2. Lặp K lần:
   - Fold thứ i làm validation set
   - K-1 folds còn lại làm training set
   - Huấn luyện và đánh giá mô hình
3. Tính trung bình của K kết quả

**Ưu điểm:**
- Sử dụng toàn bộ dữ liệu cho cả training và validation
- Giảm variance trong đánh giá
- Phát hiện overfitting/underfitting tốt hơn

**Công thức Cross-Validation Score:**

$$CV_{score} = \frac{1}{K}\sum_{i=1}^{K} Score_i$$

$$CV_{std} = \sqrt{\frac{1}{K}\sum_{i=1}^{K}(Score_i - CV_{score})^2}$$

In [ ]:
# Perform 10-Fold Cross-Validation
k_folds = 10
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

print("Performing 10-Fold Cross-Validation...")
print("="*70)

# Scale entire dataset for CV
X_scaled_full = scaler_X.fit_transform(X)

# Cross-validation for Y1
cv_r2_y1 = cross_val_score(model_y1, X_scaled_full, y1, cv=kfold, scoring='r2')
cv_mse_y1 = -cross_val_score(model_y1, X_scaled_full, y1, cv=kfold, scoring='neg_mean_squared_error')
cv_rmse_y1 = np.sqrt(cv_mse_y1)
cv_mae_y1 = -cross_val_score(model_y1, X_scaled_full, y1, cv=kfold, scoring='neg_mean_absolute_error')

print("\nY1 (Heating Load) Cross-Validation Results:")
print(f"  R² scores: {cv_r2_y1}")
print(f"  Mean R²: {cv_r2_y1.mean():.6f} (+/- {cv_r2_y1.std():.6f})")
print(f"  RMSE scores: {cv_rmse_y1}")
print(f"  Mean RMSE: {cv_rmse_y1.mean():.6f} (+/- {cv_rmse_y1.std():.6f})")
print(f"  MAE scores: {cv_mae_y1}")
print(f"  Mean MAE: {cv_mae_y1.mean():.6f} (+/- {cv_mae_y1.std():.6f})")

# Cross-validation for Y2
cv_r2_y2 = cross_val_score(model_y2, X_scaled_full, y2, cv=kfold, scoring='r2')
cv_mse_y2 = -cross_val_score(model_y2, X_scaled_full, y2, cv=kfold, scoring='neg_mean_squared_error')
cv_rmse_y2 = np.sqrt(cv_mse_y2)
cv_mae_y2 = -cross_val_score(model_y2, X_scaled_full, y2, cv=kfold, scoring='neg_mean_absolute_error')

print("\nY2 (Cooling Load) Cross-Validation Results:")
print(f"  R² scores: {cv_r2_y2}")
print(f"  Mean R²: {cv_r2_y2.mean():.6f} (+/- {cv_r2_y2.std():.6f})")
print(f"  RMSE scores: {cv_rmse_y2}")
print(f"  Mean RMSE: {cv_rmse_y2.mean():.6f} (+/- {cv_rmse_y2.std():.6f})")
print(f"  MAE scores: {cv_mae_y2}")
print(f"  Mean MAE: {cv_mae_y2.mean():.6f} (+/- {cv_mae_y2.std():.6f})")

# Store CV results for plotting
cv_results = {
    'Y1_R2': cv_r2_y1,
    'Y1_RMSE': cv_rmse_y1,
    'Y1_MAE': cv_mae_y1,
    'Y2_R2': cv_r2_y2,
    'Y2_RMSE': cv_rmse_y2,
    'Y2_MAE': cv_mae_y2
}

print("\n✓ Cross-validation completed!")

## 6. Visualizations

### 6.1 Parity Plots (Predicted vs True)

**Parity Plot (Predicted vs True Values):**

- **Trục X:** Giá trị thực tế (ground truth)
- **Trục Y:** Giá trị dự đoán (predictions)
- **Đường đỏ:** $y = x$ (dự đoán hoàn hảo)

**Cách đọc:**
- Điểm càng gần đường đỏ → dự đoán càng chính xác
- Scatter tập trung → model ổn định
- Scatter phân tán → model không ổn định

In [ ]:
# Parity Plots using loop for cleaner code
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Data for both targets
targets_data = [
    {'y_test': y1_test, 'y_pred': y1_pred_test, 'metrics': y1_test_metrics, 
     'name': 'Y1 (Heating Load)', 'color': 'tab:blue'},
    {'y_test': y2_test, 'y_pred': y2_pred_test, 'metrics': y2_test_metrics, 
     'name': 'Y2 (Cooling Load)', 'color': 'tab:green'}
]

for idx, data in enumerate(targets_data):
    ax = axes[idx]
    
    # Scatter plot
    ax.scatter(data['y_test'], data['y_pred'], alpha=0.6, s=50, 
              color=data['color'], edgecolors='k', linewidth=0.5, label='Predictions')
    
    # Perfect prediction line
    min_val, max_val = data['y_test'].min(), data['y_test'].max()
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    # Labels and title
    ax.set_xlabel(f'True {data["name"]}', fontsize=12, fontweight='bold')
    ax.set_ylabel(f'Predicted {data["name"]}', fontsize=12, fontweight='bold')
    ax.set_title(f'{data["name"]} Parity Plot\n' + 
                f'R² = {data["metrics"]["R2"]:.4f}, RMSE = {data["metrics"]["RMSE"]:.4f}',
                fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Parity plots generated!")

### 6.2 Residual Plots (Residuals vs Predicted)

**Residual Plot:**

Residual (sai số): $e_i = y_i - \hat{y}_i$

**Mục đích:** Kiểm tra giả định của Linear Regression

**Mẫu lý tưởng:**
- Residuals phân bố ngẫu nhiên xung quanh 0
- Không có pattern (hình dạng) rõ ràng
- Variance đồng đều (homoscedasticity)

**Nếu có pattern:**
- Hình cong → cần thêm polynomial features
- Hình phễu → heteroscedasticity (vi phạm giả định)
- Outliers → cần xem xét loại bỏ hoặc xử lý

In [ ]:
# Calculate residuals
residuals_data = [
    {'pred': y1_pred_test, 'residuals': y1_test - y1_pred_test, 
     'name': 'Y1 (Heating Load)', 'color': 'tab:blue'},
    {'pred': y2_pred_test, 'residuals': y2_test - y2_pred_test, 
     'name': 'Y2 (Cooling Load)', 'color': 'tab:green'}
]

# Residual plots using loop
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, data in enumerate(residuals_data):
    ax = axes[idx]
    
    # Scatter plot
    ax.scatter(data['pred'], data['residuals'], alpha=0.6, s=50,
              color=data['color'], edgecolors='k', linewidth=0.5)
    
    # Zero residual line
    ax.axhline(y=0, color='r', linestyle='--', lw=2, label='Zero Residual')
    
    # Labels and title
    ax.set_xlabel(f'Predicted {data["name"]}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Residuals', fontsize=12, fontweight='bold')
    ax.set_title(f'{data["name"]} Residual Plot\n(Residuals vs Predicted)',
                fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Residual plots generated!")

### 6.3 Residual Distribution Analysis (Histogram + Q-Q Plot)

**Kiểm tra phân phối Residuals:**

**1. Histogram:**
   - Kiểm tra xem residuals có phân phối chuẩn (normal distribution) không
   - Lý tưởng: Hình chuông đối xứng quanh 0

**2. Q-Q Plot (Quantile-Quantile Plot):**
   - So sánh quantiles của residuals với phân phối chuẩn lý thuyết
   - Lý tưởng: Các điểm nằm trên đường thẳng
   
**Giả định Linear Regression:**
$$\epsilon_i \sim N(0, \sigma^2)$$

Residuals nên tuân theo phân phối chuẩn với mean = 0 và variance = $\sigma^2$

In [ ]:
# Residual Distribution Analysis using loop
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Store residuals for both targets
y1_residuals = y1_test - y1_pred_test
y2_residuals = y2_test - y2_pred_test

residual_dist_data = [
    {'residuals': y1_residuals, 'name': 'Y1', 'color': 'skyblue', 'row': 0},
    {'residuals': y2_residuals, 'name': 'Y2', 'color': 'lightgreen', 'row': 1}
]

for data in residual_dist_data:
    row = data['row']
    
    # Histogram
    axes[row, 0].hist(data['residuals'], bins=30, edgecolor='black', 
                     alpha=0.7, color=data['color'])
    axes[row, 0].axvline(x=0, color='r', linestyle='--', lw=2, label='Zero')
    axes[row, 0].set_xlabel('Residuals', fontsize=12, fontweight='bold')
    axes[row, 0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[row, 0].set_title(f'{data["name"]} Residual Histogram', 
                          fontsize=14, fontweight='bold')
    axes[row, 0].legend()
    axes[row, 0].grid(True, alpha=0.3)
    
    # Q-Q Plot
    stats.probplot(data['residuals'], dist="norm", plot=axes[row, 1])
    axes[row, 1].set_title(f'{data["name"]} Q-Q Plot', fontsize=14, fontweight='bold')
    axes[row, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Residual distribution analysis completed!")

### 6.4 Cross-Validation Boxplots

**Boxplot của Cross-Validation Scores:**

**Mục đích:** Đánh giá độ ổn định của mô hình qua các folds

**Cách đọc:**
- **Median (đường giữa hộp):** Giá trị trung vị
- **Box (hộp):** Interquartile range (IQR) - 50% dữ liệu ở giữa
- **Whiskers (râu):** Min và Max trong khoảng chấp nhận được
- **Outliers (điểm):** Giá trị bất thường

**Đánh giá:**
- Box hẹp → Model ổn định
- Box rộng → Model không ổn định giữa các folds
- Nhiều outliers → Cần xem xét lại data hoặc model

In [ ]:
# Cross-Validation Boxplots using loop for cleaner code
fig, axes = plt.subplots(3, 3, figsize=(18, 16))

# Metrics configuration
metrics_config = [
    {'key': 'R2', 'ylabel': 'R²', 'row': 0, 'colors': ['skyblue', 'lightgreen']},
    {'key': 'RMSE', 'ylabel': 'RMSE', 'row': 1, 'colors': ['skyblue', 'lightgreen']},
    {'key': 'MAE', 'ylabel': 'MAE', 'row': 2, 'colors': ['skyblue', 'lightgreen']}
]

for config in metrics_config:
    row = config['row']
    
    # Individual boxplots for Y1 and Y2
    for col, target in enumerate(['Y1', 'Y2']):
        data_key = f'{target}_{config["key"]}'
        bp = axes[row, col].boxplot([cv_results[data_key]], labels=[target], patch_artist=True)
        bp['boxes'][0].set_facecolor(config['colors'][col])
        
        axes[row, col].set_ylabel(config['ylabel'], fontsize=12, fontweight='bold')
        axes[row, col].set_title(f'{target}: {config["ylabel"]} Cross-Validation',
                                fontsize=13, fontweight='bold')
        axes[row, col].grid(True, alpha=0.3)
        
        # Add statistics
        data = cv_results[data_key]
        axes[row, col].text(0.98, 0.98, 
                           f'Mean: {data.mean():.4f}\nStd: {data.std():.4f}',
                           transform=axes[row, col].transAxes,
                           verticalalignment='top', horizontalalignment='right',
                           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                           fontsize=9)
    
    # Comparison boxplot
    y1_data = cv_results[f'Y1_{config["key"]}']
    y2_data = cv_results[f'Y2_{config["key"]}']
    bp = axes[row, 2].boxplot([y1_data, y2_data], labels=['Y1', 'Y2'], patch_artist=True)
    
    for patch, color in zip(bp['boxes'], config['colors']):
        patch.set_facecolor(color)
    
    axes[row, 2].set_ylabel(config['ylabel'], fontsize=12, fontweight='bold')
    axes[row, 2].set_title(f'{config["ylabel"]} Comparison', fontsize=13, fontweight='bold')
    axes[row, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ All cross-validation boxplots generated (R², RMSE, MAE)!")

### 6.5 Coefficient Plot (Feature Importance)

**Coefficient Plot (Feature Importance):**

**Ý nghĩa của coefficients ($\beta_i$):**
- Độ lớn (magnitude): Mức độ ảnh hưởng của feature
- Dấu (+/-):
  - Dương (+): Feature tăng → Target tăng
  - Âm (-): Feature tăng → Target giảm

**Giải thích:**
$$\frac{\partial y}{\partial x_i} = \beta_i$$

Khi $x_i$ tăng 1 đơn vị (sau standardization), $y$ thay đổi $\beta_i$ đơn vị.

**Lưu ý:** Features đã được standardized, nên có thể so sánh trực tiếp magnitude của coefficients.

In [ ]:
# Coefficient plots using loop for cleaner code
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Models configuration
models_config = [
    {'model': model_y1, 'name': 'Y1 (Heating Load)', 'color': 'skyblue', 'idx': 0},
    {'model': model_y2, 'name': 'Y2 (Cooling Load)', 'color': 'lightgreen', 'idx': 1}
]

for config in models_config:
    ax = axes[config['idx']]
    coef = config['model'].coef_
    
    # Horizontal bar chart
    ax.barh(feature_names, coef, color=config['color'], edgecolor='black')
    ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
    
    # Labels and title
    ax.set_xlabel('Coefficient Value', fontsize=12, fontweight='bold')
    ax.set_ylabel('Features', fontsize=12, fontweight='bold')
    ax.set_title(f'{config["name"]} - Feature Coefficients\n' +
                f'Intercept: {config["model"].intercept_:.4f}',
                fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add coefficient values on bars
    for i, (name, value) in enumerate(zip(feature_names, coef)):
        ax.text(value, i, f' {value:.4f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("✓ Coefficient plots generated!")

# Print coefficient summary
print("\n" + "="*70)
print("COEFFICIENT SUMMARY")
print("="*70)
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Y1_Coefficient': model_y1.coef_,
    'Y2_Coefficient': model_y2.coef_,
    'Y1_Abs': np.abs(model_y1.coef_),
    'Y2_Abs': np.abs(model_y2.coef_)
})
coef_df = coef_df.sort_values('Y1_Abs', ascending=False)
print(coef_df[['Feature', 'Y1_Coefficient', 'Y2_Coefficient']])

## 7. Final Summary and Conclusions

**Tổng kết toàn bộ phân tích:**

Phần này tổng hợp:
1. Thông tin dataset
2. Hiệu suất mô hình (Test set và Cross-validation)
3. So sánh giữa Y1 và Y2
4. Feature quan trọng nhất
5. Đánh giá chất lượng tổng thể
6. Các biểu đồ đã tạo

**Tiêu chí đánh giá chất lượng:**
- **EXCELLENT:** R² > 0.9 cho cả 2 targets
- **VERY GOOD:** R² > 0.8 cho cả 2 targets  
- **GOOD:** R² > 0.7 cho cả 2 targets
- **MODERATE:** R² < 0.7

In [ ]:
# Final Summary Report
print("="*80)
print(" "*20 + "ENERGY EFFICIENCY PREDICTION - FINAL SUMMARY")
print("="*80)

print("\n📊 DATASET INFORMATION:")
print(f"  • Total samples: {len(df)}")
print(f"  • Training samples: {len(X_train)}")
print(f"  • Testing samples: {len(X_test)}")
print(f"  • Number of features: {X.shape[1]}")
print(f"  • Target variables: Y1 (Heating Load), Y2 (Cooling Load)")

print("\n" + "="*80)
print("🎯 Y1 (HEATING LOAD) MODEL PERFORMANCE:")
print("="*80)
print(f"  Test Set Metrics:")
print(f"    • R² Score:  {y1_test_metrics['R2']:.6f}")
print(f"    • RMSE:      {y1_test_metrics['RMSE']:.6f}")
print(f"    • MAE:       {y1_test_metrics['MAE']:.6f}")
print(f"\n  Cross-Validation (10-Fold):")
print(f"    • Mean R²:   {cv_r2_y1.mean():.6f} ± {cv_r2_y1.std():.6f}")
print(f"    • Mean RMSE: {cv_rmse_y1.mean():.6f} ± {cv_rmse_y1.std():.6f}")
print(f"    • Mean MAE:  {cv_mae_y1.mean():.6f} ± {cv_mae_y1.std():.6f}")

print("\n" + "="*80)
print("🎯 Y2 (COOLING LOAD) MODEL PERFORMANCE:")
print("="*80)
print(f"  Test Set Metrics:")
print(f"    • R² Score:  {y2_test_metrics['R2']:.6f}")
print(f"    • RMSE:      {y2_test_metrics['RMSE']:.6f}")
print(f"    • MAE:       {y2_test_metrics['MAE']:.6f}")
print(f"\n  Cross-Validation (10-Fold):")
print(f"    • Mean R²:   {cv_r2_y2.mean():.6f} ± {cv_r2_y2.std():.6f}")
print(f"    • Mean RMSE: {cv_rmse_y2.mean():.6f} ± {cv_rmse_y2.std():.6f}")
print(f"    • Mean MAE:  {cv_mae_y2.mean():.6f} ± {cv_mae_y2.std():.6f}")

print("\n" + "="*80)
print("🔍 KEY INSIGHTS:")
print("="*80)

# Determine best performing target
if y1_test_metrics['R2'] > y2_test_metrics['R2']:
    better_target = "Y1 (Heating Load)"
    better_r2 = y1_test_metrics['R2']
else:
    better_target = "Y2 (Cooling Load)"
    better_r2 = y2_test_metrics['R2']

print(f"  • Best performing target: {better_target} with R² = {better_r2:.6f}")

# Feature importance
abs_coef_y1 = np.abs(model_y1.coef_)
abs_coef_y2 = np.abs(model_y2.coef_)
most_important_y1 = feature_names[np.argmax(abs_coef_y1)]
most_important_y2 = feature_names[np.argmax(abs_coef_y2)]

print(f"  • Most important feature for Y1: {most_important_y1}")
print(f"  • Most important feature for Y2: {most_important_y2}")

# Model quality assessment
if y1_test_metrics['R2'] > 0.9 and y2_test_metrics['R2'] > 0.9:
    quality = "EXCELLENT"
elif y1_test_metrics['R2'] > 0.8 and y2_test_metrics['R2'] > 0.8:
    quality = "VERY GOOD"
elif y1_test_metrics['R2'] > 0.7 and y2_test_metrics['R2'] > 0.7:
    quality = "GOOD"
else:
    quality = "MODERATE"

print(f"  • Overall model quality: {quality}")
print(f"  • Both models show {'consistent' if abs(y1_test_metrics['R2'] - y2_test_metrics['R2']) < 0.05 else 'varying'} performance")

print("\n" + "="*80)
print("✅ CONCLUSIONS:")
print("="*80)
print("  • Linear Regression successfully predicts building energy efficiency")
print("  • High R² scores indicate strong linear relationships")
print("  • Low RMSE and MAE values confirm prediction accuracy")
print("  • Cross-validation results show model stability and generalization")
print("  • Feature coefficients reveal important building characteristics")
print("  • Model is ready for practical energy efficiency assessment")

print("\n" + "="*80)
print("📝 GENERATED VISUALIZATIONS:")
print("="*80)
print("  ✓ Parity plots (Predicted vs True) for Y1 and Y2")
print("  ✓ Residual plots (Residuals vs Predicted) for Y1 and Y2")
print("  ✓ Residual histograms for Y1 and Y2")
print("  ✓ Q-Q plots for residual normality check")
print("  ✓ Cross-validation boxplots for R², RMSE, and MAE")
print("  ✓ Coefficient plots showing feature importance")

print("\n" + "="*80)
print("🎉 LINEAR REGRESSION ANALYSIS COMPLETED SUCCESSFULLY!")
print("="*80)